# United States - Crime Rates - 1960 - 2014

### Introduction:

This time you will create a data 

Special thanks to: Karthik, A brilliant tTutor/expert on finding revenue of chipo
### Step 1. Import the necessary libraries

In [0]:
import pandas as pd
import numpy as np

### Step 2.  US_Crime_Rates_1960_2014

In [0]:
df = spark.read.csv("/mnt/adventureworks2022/tsvfiles/US_Crime_Rates_1960_2014.csv", inferSchema=True, header=True)

### Step 3. Assign it to a variable called crime.

In [0]:
crimes=df.toPandas()
crimes.head(10)

### Step 4. What is the type of the columns?

In [0]:
crimes.info()

##### Have you noticed that the type of Year is int64. But pandas has a different type to work with Time Series. Let's see it now.

### Step 5. Convert the type of the column Year to datetime64

In [0]:
crimes_Year_DateTime = crimes
crimes_Year_DateTime['Year']=pd.to_datetime(crimes_Year_DateTime['Year'],format="%Y")
crimes_Year_DateTime.head(10)

### Step 6. Set the Year column as the index of the dataframe

In [0]:
crimes.set_index('Year')

### Step 7. Delete the Total column

In [0]:
del crimes['Total']
crimes.head(2)

### Step 8. Group the year by decades and sum the values

#### Pay attention to the Population column number, summing this column is a mistake

In [0]:
crimes_Year_Y=crimes
crimes_Year_Y['Year'] = pd.to_datetime(crimes_Year_Y.Year,format='%Y')


In [0]:
period_length=10
start_year=1960
end_year=2014
year_range = end_year - start_year

modulo = year_range % period_length

if modulo == 0:
    final_start = end_year - period_length
else:
    final_start = end_year - modulo
final_end = end_year + 1

starts = np.arange(start_year, final_start, period_length).tolist()

tuples = [(start, start+period_length) for start in starts]
# We'll add the last period calculated earlier
tuples.append(tuple([final_start, final_end]))
bins = pd.IntervalIndex.from_tuples(tuples, closed='left')

original_labels = list(bins.astype(str))
new_labels = ['{} - {}'.format(b.strip('[)').split(', ')[0], int(b.strip('[)').split(', ')[1])-1) for b in original_labels]
label_dict = dict(zip(original_labels, new_labels))

#Assign each row to a period
crimes_Year_Y['PERIOD'] = pd.cut(crimes_Year_Y['YEAR'], bins=bins, include_lowest=True, precision=0)
crimes_Year_Y['PERIOD'] = crimes_Year_Y['PERIOD'].astype("str")
crimes_Year_Y = crimes_Year_Y.replace(label_dict)


crimes_Year_Y = crimes_Year_Y.groupby('PERIOD').mean().reset_index().drop('YEAR', axis=1)

### Step 9. What is the most dangerous decade to live in the US?